In [1]:
library(tidyverse)
library(WGCNA)
library(cowplot)
library(matrixStats)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: dynamicTreeCut
Loading required package: fastcluster

Attaching package: ‘fastcluster’

The following object is masked from ‘package:stats’:

    hclust



Attaching package: ‘WGCNA’

The following object is masked from ‘package:stats’:

    cor


********************************************************
Note: As of version 1.0.0, cowplot does not change the
  default ggplot2 theme anymore. To recover the previous
  behavior, execute:
  theme_set(theme_cowplot())
********************************************************


Attaching package: ‘matrixStats’

The following object is masked 

In [2]:
options(stringsAsFactors = FALSE)
enableWGCNAThreads()

Allowing parallel execution with up to 15 working processes.


In [3]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

event_code <- list("Alive" = 0, "Dead" = 1)

In [4]:
dset_idx <- 3

# Load data

In [5]:
clinical_df <- load_survival_df(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/survival_data.tsv"), event_code)
data_traits <- clinical_df %>%
    dplyr::select(sample_name, vital_status, survival_time, figo_stage) %>%
    dplyr::filter(rowSums(is.na(.)) == 0) %>% 
    decode_figo_stage(to = "n") %>%
    dplyr::mutate(figo_stage = as.character(figo_stage)) %>%
    dplyr::bind_cols(to_one_hot(., col = "figo_stage")) %>%
    dplyr::select(-figo_stage) %>%
    column_to_rownames("sample_name")

Parsed with column specification:
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)


In [6]:
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_cancer_network.RData"))

In [7]:
n_genes <- ncol(data_expr)
n_samples <- nrow(data_expr)

In [8]:
mes_0 <- moduleEigengenes(data_expr, module_colors)$eigengenes
mes <- orderMEs(mes_0)

# Module significance for survival analysis

In [9]:
module_coxph_res <- standardScreeningCensoredTime(
    datExpr = mes,
    time = data_traits$survival_time, 
    event = data_traits$vital_status, 
    fastCalculation = FALSE
)

In [10]:
module_coxph_res

ID,pvalueWald,qvalueWald,pvalueLogrank,qvalueLogrank,pvalueDeviance,qvalueDeviance,corDeviance,HazardRatio,CI.LowerLimitHR,CI.UpperLimitHR,C.index
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MEblue,0.6047159,0.4974559,0.6048581,0.4973085,0.6788790,0.6195655,0.035484726,4.137923910,1.911376e-02,895.816223,0.4675180
MEblack,0.6327989,0.4974559,0.6326113,0.4973085,0.4865288,0.5718553,-0.059597114,0.296177043,2.012537e-03,43.587195,0.5607996
MEgreen,0.9599033,0.6173994,0.9599033,0.6173994,0.9072445,0.6774371,-0.009990736,0.890672456,9.759841e-03,81.281801,0.4508606
MEmagenta,0.1778580,0.4974559,0.1743376,0.4973085,0.1468638,0.4788399,0.123758674,14.244052483,2.988622e-01,678.884780,0.4491949
MEpurple,0.7519023,0.5319770,0.7519430,0.5320057,0.8284264,0.6774371,0.018581461,2.116662005,2.025963e-02,221.142133,0.4675180
MEbrown,0.3236560,0.4974559,0.3227801,0.4973085,0.2188142,0.4788399,0.105055757,9.562845165,1.079613e-01,847.044576,0.3936702
MEred,0.6225988,0.4974559,0.6222986,0.4973085,0.4873566,0.5718553,-0.059484232,0.293414732,2.222303e-03,38.740079,0.5574681
MEpink,0.1137725,0.4974559,0.1176336,0.4973085,0.1599449,0.4788399,-0.119919924,0.003649827,3.475765e-06,3.832606,0.5685730
MEturquoise,0.2474019,0.4974559,0.2466300,0.4973085,0.2331917,0.4788399,0.101873739,16.166459817,1.447493e-01,1805.566435,0.4025541


# Module significance for FIGO stage

In [11]:
module_figo_cor <- WGCNA::cor(mes, data_traits[-c(1:2)])
module_figo_cor

,figo_stage_1,figo_stage_2,figo_stage_3,figo_stage_4
MEblue,0.067777692,-0.050861998,-0.006235733,-0.06193354
MEblack,0.088360268,0.140937783,-0.150530048,-0.11373487
MEgreen,-0.012960337,0.159038968,-0.093492239,-0.02817891
MEmagenta,-0.125442120,-0.125013774,0.192129061,0.08786297
MEpurple,-0.113737551,0.008597862,0.131790896,-0.02476229
MEbrown,-0.219056181,-0.058215461,0.213612209,0.15238589
MEred,0.116769308,0.129756827,-0.177800615,-0.10496292
MEpink,-0.008464123,0.103872006,-0.032029183,-0.07805203
MEturquoise,-0.176822631,0.113499127,0.135664526,-0.05356945
MEyellow,-0.090777561,0.071747470,0.072506900,-0.05454905


In [12]:
module_figo_pvals <- corPvalueStudent(module_figo_cor, n_samples)
module_figo_pvals

,figo_stage_1,figo_stage_2,figo_stage_3,figo_stage_4
MEblue,0.427903990,0.55209473,0.94192149,0.46888917
MEblack,0.300963365,0.09794517,0.07692894,0.18248724
MEgreen,0.879639161,0.06147963,0.27364665,0.74193662
MEmagenta,0.141181818,0.14255568,0.02345959,0.30370268
MEpurple,0.182476880,0.91998407,0.12198041,0.77231268
MEbrown,0.009573668,0.49604241,0.01157238,0.07331679
MEred,0.171016185,0.12789893,0.03625787,0.21880211
MEpink,0.921224680,0.22365413,0.70817773,0.36108249
MEturquoise,0.037313238,0.18340150,0.11129739,0.53110429
MEyellow,0.287880331,0.40128190,0.39630348,0.52360953


In [13]:
module_figo_padjs <- apply(module_figo_pvals, MARGIN = 2, function(x) { p.adjust(x, method = "BH") })
module_figo_padjs

,figo_stage_1,figo_stage_2,figo_stage_3,figo_stage_4
MEblue,0.5229938,0.6073042,0.9419215,0.6491275
MEblack,0.4138246,0.3136225,0.1692437,0.6017058
MEgreen,0.9212247,0.3136225,0.3762641,0.7723127
MEmagenta,0.3345409,0.3136225,0.1290277,0.6491275
MEpurple,0.3345409,0.9199841,0.1916835,0.7723127
MEbrown,0.1053103,0.6062741,0.1272962,0.4032424
MEred,0.3345409,0.3136225,0.1303341,0.6017058
MEpink,0.9212247,0.3514565,0.7789955,0.6491275
MEturquoise,0.2052228,0.3362361,0.1916835,0.6491275
MEyellow,0.4138246,0.5517626,0.4843709,0.6491275
